# Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import os, sys
from importlib import reload
from config import data_config
from utils.args_parser import add_data_args

# Reload arg_parser because Jupyter notebook does not reload it automatically
reload(data_config)

# Add arguments to parser
parser = ArgumentParser()
parser = add_data_args(parser)

# Print default dataset parameters
args = parser.parse_args([])
print("\n=====================")
print("Default dataset parameters:")
for arg in vars(args):
    print(arg, '=', getattr(args, arg))
print("\n=====================")




Default dataset parameters:
ode_system = pendulum
t_max = 20
step_size = 0.01
n_sample = 10
x_init_pts = [[-3.14159265  3.14159265]
 [-8.          8.        ]]
ctr_func = gaussian
datafile_path = data/pendulum_ctr_gaussian_N_10_T20.npy



In [20]:
# Run database_generator.py with arguments
print("\n=====================")
print("Running database_generator.py with arguments:")
%run database_generator.py --n_sample 1 --ctr_func "gaussian" --t_max 10 --datafile_path "data/pendulum_ctr_grf_N_1_T10.npy"



Running database_generator.py with arguments:


Generating 1 data ...: 100%|██████████| 1/1 [00:00<00:00,  4.56it/s]

Data saved in data/pendulum_ctr_grf_N_1_T10.npy .


In [1]:
%run train.py --datafile_path "data/pendulum_ctr_grf_N_5000_T10.npy" 

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using all available 1 GPUs.
Shapes for LSTM-MIONet training input=(18000, 999, 1), x_n=(18000, 1), x_next=(18000, 1)
Shapes for LSTM-MIONet training input=(2000, 999, 1), x_n=(2000, 1), x_next=(2000, 1)
Data memory size: 68 MB
Data memory size: 7 MB
LSTM_MIONet_Static(
  (layernorm): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
  (layernorm_x_n): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
  (layernorm_delta_t): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
  (lstm): LSTM(150, 10, num_layers=2, batch_first=True)
  (cell_mlp): Sequential(
    (0): Linear(in_features=10, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=150, bias=True)
  )
  (input_mlp): Sequential(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=150, bias=True)
  )
  (delta_t_mlp): Sequential(
    (0): Lin

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
  3%|▎         | 31/1000 [04:47<2:24:44,  8.96s/it, Train=0.677, Val=0.163, Best_train=0.186, Best_Val=0.0899]

Epoch 00031: reducing learning rate of group 0 to 5.0000e-04.


  6%|▌         | 62/1000 [09:28<2:13:42,  8.55s/it, Train=0.0946, Val=0.0809, Best_train=0.0663, Best_Val=0.052] 

...

Epoch 00113: reducing learning rate of group 0 to 3.1250e-05.


 13%|█▎        | 127/1000 [19:02<2:04:14,  8.54s/it, Train=0.048, Val=0.049, Best_train=0.0465, Best_Val=0.0464]  

Epoch 00127: reducing learning rate of group 0 to 1.5625e-05.


 15%|█▌        | 150/1000 [22:34<2:07:55,  9.03s/it, Train=0.046, Val=0.0474, Best_train=0.0454, Best_Val=0.0456] 


KeyboardInterrupt: 

In [17]:
import pandas as pd
import numpy as np

datafile_path = "/PENDULUM/archive/data/pendulum_u_random_init_a_001.pkl"
data = pd.read_pickle(datafile_path)
print(type(data["u"]))
print(data["u"].shape)
print(data["theta"].shape)
print(data["omega"].shape)
print(data["t"].shape)
data_new = dict()
data_new["u"] = np.expand_dims(data["u"].T, axis=1)
data_new["x"] = np.dstack((data["theta"].T, data["omega"].T)).transpose(0, 2, 1)
data_new["t"] = data["t"]
print(data_new["u"].shape)
print(data_new["x"].shape)
idxs_all = np.arange(data_new["u"].shape[2])
idxs_nan_u = np.isnan(data_new["u"]).sum(axis=(0,1)) 
idxs_nan_x = np.isnan(data_new["x"]).sum(axis=(0,1))
idxs_nan = idxs_nan_u + idxs_nan_x
idxs_valid = idxs_all[idxs_nan==0]
print(idxs_all.shape)
print(idxs_valid.shape)
data_new["u"] = data_new["u"][:, :, idxs_valid]
data_new["x"] = data_new["x"][:, :, idxs_valid]
np.save("/PENDULUM/src/data/pendulum_ctr_grf_N_5000_T10_2.npy", data_new)

<class 'numpy.ndarray'>
(5000, 999)
(5000, 999)
(5000, 999)
(999, 1)
(999, 1, 5000)
(999, 2, 5000)
(5000,)
(4820,)


In [19]:
import pandas as pd
import numpy as np

datafile_path = "/PENDULUM/archive/data/pendulum_u_test_random_init_a_001_stat.pkl"
data = pd.read_pickle(datafile_path)
print(type(data["u"]))
print(data["u"].shape)
print(data["theta"].shape)
print(data["omega"].shape)
print(data["t"].shape)
data_new = dict()
data_new["u"] = np.expand_dims(data["u"].T, axis=1)
data_new["x"] = np.dstack((data["theta"].T, data["omega"].T)).transpose(0, 2, 1)
data_new["t"] = data["t"]
print(data_new["u"].shape)
print(data_new["x"].shape)
idxs_all = np.arange(data_new["u"].shape[2])
idxs_nan_u = np.isnan(data_new["u"]).sum(axis=(0,1)) 
idxs_nan_x = np.isnan(data_new["x"]).sum(axis=(0,1))
idxs_nan = idxs_nan_u + idxs_nan_x
idxs_valid = idxs_all[idxs_nan==0]
print(idxs_all.shape)
print(idxs_valid.shape)
data_new["u"] = data_new["u"][:, :, idxs_valid]
data_new["x"] = data_new["x"][:, :, idxs_valid]
np.save("/PENDULUM/src/data/pendulum_ctr_grf_N_100_T10_2.npy", data_new)


<class 'numpy.ndarray'>
(120, 999)
(120, 999)
(120, 999)
(999, 1)
(999, 1, 120)
(999, 2, 120)
(120,)
(115,)


In [1]:
import pandas as pd
import numpy as np
from utils.nn_lib import Customize_Dataset
import torch
from torch.utils.data import DataLoader, random_split
from utils.data_utils import split_dataset, prepare_local_predict_dataset, scale_and_to_tensor, Dataset_Stat, Dataset_Torch
from config import train_config

seed = 999
np.random.seed(seed)
torch.manual_seed(seed)

datafile_path = "/PENDULUM/archive/data/pendulum_u_test_random_init_a_001_stat.pkl"
config = dict()
config.update(train_config.get_config())

dataset_o = Customize_Dataset(
        filepath=datafile_path,
        search_len=10,
        search_num=20,
        use_padding=True,
        search_random = True,
        device=torch.device("cpu"),
        transform=None,
        target_transform=None,
    )

print(dataset_o.metadata[0:20,-2])

datafile_path = "/PENDULUM/src/data/pendulum_ctr_grf_N_100_T10_2.npy"
dataset = np.load(datafile_path, allow_pickle=True).item()
data, _ = split_dataset(
        dataset, test_size=0., verbose=False
    )
input_masked, x_n, x_next, t_params = prepare_local_predict_dataset(
        data=data,
        state_component=config["state_component"],
        t_max=config["t_max"],
        search_len=config["search_len"],
        search_num=config["search_num"],
        search_random=config["search_random"],
        offset=config["offset"],
        verbose=config["verbose"],
    )
data_stat = Dataset_Stat(input_masked, x_n, x_next, t_params)
input_masked, x_n, x_next, t_params = scale_and_to_tensor(
        data_stat, scale_mode=config["scale_mode"]
    )
data_torch = Dataset_Torch(input_masked, x_n, x_next, t_params)
print(data_torch.x_next[0:20,0])

Data shape: torch.Size([2300, 999])
Data memory: 8.82 MB
tensor([-56.0552,   0.7273,  -0.7407,  -2.2929, -11.3510,   1.6141,  12.0482,
         -0.8410,  -1.5140,   1.2153,  -0.6183,  23.9170,  -0.1437,   0.6723,
        -41.4200,  -0.4753, -47.2333,  -2.3165,  -0.2414,   1.7439])
Shapes for LSTM-MIONet training input=(2300, 999, 1), x_n=(2300, 1), x_next=(2300, 1)
Data memory size: 8 MB
tensor([-56.0552,   0.7273,  -0.7407,  -2.2929, -11.3510,   1.6141,  12.0482,
         -0.8410,  -1.5140,   1.2153,  -0.6183,  23.9170,  -0.1437,   0.6723,
        -41.4200,  -0.4753, -47.2333,  -2.3165,  -0.2414,   1.7439])
